# GIS 5571 - Lab 1
### Luke Zaruba
##### September 27, 2022

In [1]:
# Import Libraries
import requests
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point, LineString, shape
import json

/Users/lukezaruba/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# 1. Google Places API ETL Pipeline

### Functions

In [2]:
def searchGoogle(lat, long, radius, place_type, keyword, api_key):
    # Replacing URL substrings with search parameters
    google_places = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=LOC&radius=RAD&type=TYPE&keyword=KW&key=YOUR_API_KEY"

    location = str(lat) + "%2C" + str(long)

    google_places = google_places.replace("LOC", location)
    google_places = google_places.replace("RAD", str(radius))
    google_places = google_places.replace("TYPE", place_type)
    google_places = google_places.replace("KW", keyword)
    google_places = google_places.replace("YOUR_API_KEY", api_key)

    # Getting request
    payload={}
    headers = {}

    response = requests.request("GET", google_places, headers=headers, data=payload)

    # Note: Not an actual JSON - can keep to use Python list/dist methods for parsing
    json_response = response.json()

    return json_response["results"]

In [3]:
def googleJsonToDf(json_input):
    # Create DF with column names
    df = pd.DataFrame(columns=("Name", "Latitude", "Longitude", "Rating", "Address", "Icon_URL"))
    
    # Loop through the JSON and add all features to the DF
    for i in range(len(json_input)):
        name = [json_input[i]["name"]]
        lat = [json_input[i]["geometry"]["location"]["lat"]]
        lng = [json_input[i]["geometry"]["location"]["lng"]]
        rating = [json_input[i]["rating"]]
        address = [json_input[i]["vicinity"]]
        icon = [json_input[i]["icon"]]
        
        df.loc[i] = name + lat + lng + rating + address + icon
    
    # Convert DF to GDF, set CRS, and return
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
    
    gdf.set_crs("EPSG:4326")
        
    return gdf

In [4]:
def makeGoogleMap(lat, long, zoom, gdf):
    # Create an empty base map in Folium
    my_map = folium.Map(location=[lat, long], tiles="Cartodb Positron", zoom_start=zoom)
    
    # Create list of point gemoetries from GDF
    point_list = [[pt.xy[1][0], pt.xy[0][0]] for pt in gdf.geometry]
    
    # Loop though point list, add markers to map (w/ Google symbology), and returning map
    i = 0
    
    for feature in point_list:
        custom_icon = folium.features.CustomIcon(gdf.Icon_URL[i], icon_size=(14, 14))
        
        my_map.add_child(folium.Marker(location = feature,
                            popup =
                            "Name: " + str(gdf.Name[i]) + '\n' +
                            "Address: " + str(gdf.Address[i]) + '\n' +
                            "Rating: " + str(gdf.Rating[i]) + '\n' +
                            "Coordinates: " + str(point_list[i]),
                            icon = custom_icon))
        i += 1
        
    return my_map

### Running Functions

In [5]:
key = "YOUR_API_KEY_HERE"

google_query = searchGoogle(44.971635, -93.270123, "100000", "", "charging station", key)

In [6]:
charging_stations = googleJsonToDf(google_query)

In [7]:
makeGoogleMap(44.971635, -93.270123, 12, charging_stations)

# Minnesota Geospatial Commons ETL Pipeline

### Functions

In [8]:
def searchMnGeo(query_url, outSR = "4326"):
    # Base parameters for REST API query
    params = f"?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=&returnGeometry=true&maxAllowableOffset=&geometryPrecision=&outSR={outSR}&havingClause=&gdbVersion=&historicMoment=&returnDistinctValues=false&returnIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&multipatchOption=xyFootprint&resultOffset=&resultRecordCount=&returnTrueCurves=false&returnExceededLimitFeatures=false&quantizationParameters=&returnCentroid=false&sqlFormat=none&resultType=&featureEncoding=esriDefault&datumTransformation=&f=json"
    
    # Join URL with parameters and request the data
    url = query_url + params
    
    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)

    # Return as an actual JSON (different approach from Google Places ETL function)
    json_response = json.dumps(response.json())
    
    # Read in JSON, set CRS, and return a GDF
    gdf = gpd.read_file(json_response)
    
    gdf.set_crs("EPSG:4326")
    
    return gdf

In [9]:
def makeMnGeoMap(lat, long, zoom, gdf):
    # Create an empty base map in Folium
    my_map = folium.Map(location=[lat, long], tiles="Cartodb Positron", zoom_start=zoom)
    
    # Loop through rows and convert to JSON (Folium does not directly support GDFs)
    for i, row in gdf.iterrows():
        simplified = gpd.GeoSeries(row['geometry']).simplify(tolerance=0.001)
        geojson = simplified.to_json()
        flm_geo = folium.GeoJson(data=geojson, style_function=lambda x: {"fillOpacity": 0.2, 'fillColor': 'blue'})
        flm_geo.add_to(my_map)
        
    return my_map

### Running Functions

In [10]:
base_url = 'https://arcgis.metc.state.mn.us/server/rest/services/GISLibrary/CTUs/FeatureServer/0/query'

tcma_ctu = searchMnGeo(base_url)

In [11]:
makeMnGeoMap(44.971635, -93.270123, 9, tcma_ctu)

# NDAWN ETL Pipeline

### Functions

In [12]:
def searchNDAWN(station_id, begin_date, end_date):
    # Base URL
    url = "https://ndawn.ndsu.nodak.edu/table.csv?station=ID&variable=ddmxt&variable=ddmxtt&variable=ddmnt&variable=ddmntt&variable=ddavt&variable=dddtr&variable=ddbst&variable=ddtst&variable=ddws&variable=ddmxws&variable=ddmxwst&variable=ddwd&variable=ddwdsd&variable=ddsr&variable=ddtpetp&variable=ddtpetjh&variable=ddr&variable=dddp&variable=ddwc&variable=ddmnwc&variable=ddmxt9&variable=ddmxtt9&variable=ddmnt9&variable=ddmntt9&variable=ddmxws10&variable=ddmxwst10&variable=ddwd10&variable=ddwdsd10&year=2022&ttype=daily&quick_pick=&begin_date=BEGDATE&end_date=ENDATE"
    
    # Enter in parameters (dates must be of following format: 2022-09-19)
    url = url.replace("ID", str(station_id))
    url = url.replace("BEGDATE", str(begin_date))
    url = url.replace("ENDATE", str(end_date))
    
    # Request data and return output of CSV file
    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    
    file_name = f"ndawn_{str(station_id)}.csv"
    
    csv = open(file_name,'w')
    csv.write(response.text)
    csv.close()
    
    # Take CSV and read into DF
    df = pd.read_csv(file_name, header=3, skiprows=[4])
    
    columns = ['Station Name', 'Latitude', 'Longitude', 'Elevation', 'Year', 'Month', 'Day', 'Max Temp', 'Min Temp', 'Avg Temp', 'Avg Wind Speed', 'Total Solar Rad', 'Rainfall', 'Dew Point']
    
    final_df = df[columns].copy()
    
    # Convert DF to GDF, set CRS, and return
    gdf = gpd.GeoDataFrame(final_df, geometry=gpd.points_from_xy(final_df.Longitude, final_df.Latitude))
    
    gdf.set_crs("EPSG:4326")
    
    return gdf

### Running Functions

In [13]:
ndawn_data = searchNDAWN("95", "2022-09-19", "2022-09-22")

ndawn_data.head()

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Max Temp,Min Temp,Avg Temp,Avg Wind Speed,Total Solar Rad,Rainfall,Dew Point,geometry
0,Williams,48.858419,-94.980798,1093,2022,9,19,74.048,48.056,61.052,5.811,327.258,0.000,55.649,POINT (-94.98080 48.85842)
1,Williams,48.858419,-94.980798,1093,2022,9,20,69.044,51.098,60.071,8.469,266.658,0.008,53.356,POINT (-94.98080 48.85842)
2,Williams,48.858419,-94.980798,1093,2022,9,21,57.884,47.012,52.448,12.714,309.691,0.020,42.542,POINT (-94.98080 48.85842)
3,Williams,48.858419,-94.980798,1093,2022,9,22,56.300,37.787,47.044,4.364,406.425,0.000,38.539,POINT (-94.98080 48.85842)


# Running Spatial Join

In [14]:
def showSpatialJoin(lat, long, zoom, gdf):
    # Create an empty base map in Folium
    my_map = folium.Map(location=[lat, long], tiles="Cartodb Positron", zoom_start=zoom)
    
    # Create list of point gemoetries from GDF
    point_list = [[pt.xy[1][0], pt.xy[0][0]] for pt in gdf.geometry]
    
    # Loop though point list, add markers to map (w/ Google symbology), and returning map
    i = 0
    
    for feature in point_list:
        custom_icon = folium.features.CustomIcon(gdf.Icon_URL[i], icon_size=(14, 14))
        
        my_map.add_child(folium.Marker(location = feature,
                            popup =
                            "Name: " + str(gdf.Name[i]) + '\n' +
                            "City from Join: " + str(gdf.CTU_NAME[i]),
                            icon = custom_icon))
        i += 1
        
    return my_map

In [15]:
chargingstation_cities = gpd.sjoin(charging_stations.set_crs("EPSG:4326"), tcma_ctu)

In [16]:
showSpatialJoin(44.971635, -93.270123, 12, chargingstation_cities)

In [17]:
chargingstation_cities.head()

,Name,Latitude,Longitude,Rating,Address,Icon_URL,geometry,index_right,CTU_NAME
0,Electric Vehicle Charging Station,44.973878,-93.230153,4.3,"272 SE Harvard St, Minneapolis",https://maps.gstatic.com/mapfiles/place_api/ic...,POINT (-93.23015 44.97388),54,Minneapolis
1,Electric Vehicle Charging Station,44.975125,-93.253605,1.0,"1023b S Washington Ave, Minneapolis",https://maps.gstatic.com/mapfiles/place_api/ic...,POINT (-93.25360 44.97512),54,Minneapolis
2,Tesla Destination Charger,44.976151,-93.269278,5.0,"618 2nd Ave S, Minneapolis",https://maps.gstatic.com/mapfiles/place_api/ic...,POINT (-93.26928 44.97615),54,Minneapolis
3,ChargePoint Charging Station,44.974089,-93.231696,0.0,"511 Washington Ave SE, Minneapolis",https://maps.gstatic.com/mapfiles/place_api/ic...,POINT (-93.23170 44.97409),54,Minneapolis
4,SemaConnect Charging Station,44.969081,-93.280713,0.0,"1369 Spruce Pl, Minneapolis",https://maps.gstatic.com/mapfiles/place_api/ic...,POINT (-93.28071 44.96908),54,Minneapolis


# Converting Pandas DataFrame to Feature Class